# <span style="color:darkblue"> Lecture 16 - Practicing Chaining </span>

<font size = "5">

In this lecture you will get a chance to practice <br>
the main dataset operations

- There will be a quiz on this lecture

# <span style="color:darkblue"> I. Import Libraries and Data </span>


In [23]:
import numpy as np
import pandas as pd

In [24]:
results  = pd.read_csv("data_raw/results.csv")
races    = pd.read_csv("data_raw/races.csv")
results["points col"] = results["points"]

# <span style="color:darkblue"> II. Review Dataset Operations </span>

<font size = "5">

See attached file for a refresher on syntax

```[] ``` $\qquad \qquad \qquad \quad$: Extracting columns <br>
```.query() ``` $\qquad \qquad $: Subsetting rows <br>
```.recode() ``` $ \qquad \quad \ \ $: Replacing values <br>
```.groupby().agg() ```: Aggregate statistics by subgroup <br>
```.rename() ``` $\qquad \quad \ \ $: Change name of columns

Full list:

<font size = "4">

https://www.w3schools.com/python/pandas/pandas_ref_dataframe.asp

# <span style="color:darkblue"> III. Examples of Chaining </span>

<font size = "5">

The operations with "." are read left to right

- Combine any of the above operations
- Great way to make code efficient
- The sky's the limit!


Subsetting **before** extracting columns

In [25]:
# Get data for drivers that scored more than 20 points on individual races
# Then extract the columns "driverId" and "points"
results.query('points >= 20')[["driverId","points"]]

,driverId,points
20320,4,25.0
20344,18,25.0
20368,20,25.0
20392,18,25.0
20416,17,25.0
...,...,...
25740,830,25.0
25760,830,25.0
25780,830,25.0
25800,847,26.0


<font size = "5">

Subsetting **before** aggregating

In [26]:
# This obtains a subset of drivers who competed in races 500 onwards
# then computes the average by team ("constructorId")

(results.query('raceId >= 500')
        .groupby("constructorId")
        .agg(mean_points = ("points","mean")))


,mean_points
constructorId,
1,3.148148
3,1.904924
4,1.903226
5,1.203911
6,4.910966
...,...
209,0.012821
210,0.809028
211,3.723684


<font size = "5">

Subsetting **after** aggregating

In [27]:
# This obtains the average points by team ("constructorId"), then 
# produces a subset of team whose average is higher than 10

(results.groupby("constructorId")
        .agg(mean_points = ("points","mean"))
        .query('mean_points >= 10'))

,mean_points
constructorId,
131,12.363643


<font size = "5">

Chaining inside queries + NaNs

In [28]:
# "is.na()" produces a True/False vector, checking for missing values
# "is.notna()" produces a True/False vector, checking for non-missing values
# .str.isnumeric()
results["points"].isna()
results["points"].notna()

subset_nas    = results.query('points.isna()')
subset_nonnas = results.query('points.notna()')


## <span style="color:darkblue"> III. Quiz Structure </span>

<font size = "5">

The day of the quiz I will ...
- Provide a dataset with information
- Give more specific instructions.
- Below, you will see the type of questions that will be asked.
- The idea is for you to apply known concepts to new data
- You have 50 minutes to complete the assignment

Questions

(exact wording may change in quiz, but exercise will be very similar)



<font size = "5">

(a) Replace the values of a column

- Obtain unique string values of a column
- Use the ".replace()" command

Hint: See Lecture 13

In [29]:

subset      = results.query("milliseconds.str.isnumeric() == False")
list_unique = pd.unique(subset["milliseconds"])
print(list_unique)
list_old = ['\\N']
list_new = [np.nan]
results["milliseconds"] = results["milliseconds"].replace(list_old, list_new)
results["milliseconds_numeric"] = pd.to_numeric(results["milliseconds"])

['\\N']


<font size = "5">

(b) Recode a numeric column

- Use the "pd.cut()" command to create <br>
a new column based on an interval.
- See Lecture 14 for more details

In [40]:
circuits = pd.read_csv("data_raw/circuits.csv")

subset      = circuits.query("alt.str.isnumeric() == False")
list_unique = pd.unique(subset["alt"])
print(list_unique)
list_old = ['\\N','-7']
list_new = [np.nan,-7]
circuits["alt"] = circuits["alt"].replace(list_old, list_new)

bins_x = [0,2500, 5000]
labels_x = ["Between 0 and 2500",
            "Between 2500 and 5000"]
circuits["alt_numeric"] = pd.to_numeric(circuits["alt"])


circuits["bins_alt"] = pd.cut(circuits["alt_numeric"],
                              bins = bins_x,
                              right = True,
                              labels = labels_x)

['\\N' '-7']


<font size = "5">

(c) Aggregate and query

- Use a combniation of the following commands <br>
to produce a new dataset <br>
``` .query() ``` <br>
``` .groupby().agg() ``` <br>

In [ ]:
results_agg2 = (results.query("raceId >= 200")
                       .groupby(["constructorId"])
                        .agg(mean_laps = ('laps','mean'),
                             sd_laps =   ('laps','std'),))
results_agg2


<font size = "5">

(d) Aggregate and sort

- Use a combniation of the following commands <br>
to produce a new dataset <br>
``` .groupby().agg() ``` <br>
``` .sort_values() ```

In [ ]:
results_agg = (results.groupby("constructorId")
               .agg(meanlaps = ('laps', 'mean'),
                    stdvlaps = ('laps', 'std'))
                    .sort_values(by = "meanlaps", ascending=False))
results_agg
               

<font size = "5">

(e) Rename column

- Create a dictionary
- Rename one or more columns in a dataset <br>
using the dictionary

In [42]:
dict_rename_circuits = {"name": "circuit_name"}
circuits = circuits.rename(columns = dict_rename_circuits)

<font size = "5">

(f) Merge dataset

- Use "pd.merge" to combine two datasets: <br>
a primary and secondary
- Only merge a subset of the columns of the <br>
secondary dataset
- Use "display" to show a the merged dataset,  <br>
extracting a subset of the columns

In [43]:
races_merge = pd.merge(races,
                       circuits[["circuitId","circuit_name"]],
                       on = "circuitId",
                       how = "left")
display(races_merge[["raceId", "circuitId", "circuit_name"]].sort_values(by = "circuitId"))
print(races_merge.columns.values)

,raceId,circuitId,circuit_name
0,1,1,Albert Park Grand Prix Circuit
54,55,1,Albert Park Grand Prix Circuit
70,71,1,Albert Park Grand Prix Circuit
858,860,1,Albert Park Grand Prix Circuit
89,90,1,Albert Park Grand Prix Circuit
...,...,...,...
1096,1115,78,Losail International Circuit
1038,1051,78,Losail International Circuit
1083,1102,79,Miami International Autodrome
1061,1078,79,Miami International Autodrome


['raceId' 'year' 'round' 'circuitId' 'name' 'date' 'time' 'url' 'fp1_date'
 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time' 'quali_date'
 'quali_time' 'sprint_date' 'sprint_time' 'circuit_name']
